**Import Libraries**

In [83]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType # for creating schema
from pyspark.sql.functions import lit # Lit is used to add new columns to a dataframe in spark

In [30]:
# Create SparkSession
spark = SparkSession.builder.getOrCreate()

# Check Spark version
print("Spark version:", spark.version)

Spark version: 4.0.0


In [32]:
# Loading the file
df = pd.read_csv('US_crimes_2024.csv')
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13709672,JJ101940,12/31/2024 11:58:00 PM,014XX E 68TH ST,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,...,5,43.0,14,1186817.0,1860189.0,2024,05/17/2025 03:40:52 PM,41.771470,-87.590742,POINT (-87.59074212 41.771470188)
1,13707925,JJ100089,12/31/2024 11:56:00 PM,047XX S DR MARTIN LUTHER KING JR DR,1365,CRIMINAL TRESPASS,TO RESIDENCE,APARTMENT,True,True,...,3,38.0,26,1179661.0,1873623.0,2024,05/17/2025 03:40:52 PM,41.808501,-87.616563,POINT (-87.616562762 41.808500903)
2,13708038,JJ100035,12/31/2024 11:55:00 PM,077XX S CICERO AVE,0498,BATTERY,"AGG. DOMESTIC BATTERY - HANDS, FISTS, FEET, SE...",HOTEL / MOTEL,False,True,...,18,70.0,04B,1145740.0,1853048.0,2024,05/17/2025 03:40:52 PM,41.752749,-87.741498,POINT (-87.741497836 41.752748627)
3,13709164,JJ101392,12/31/2024 11:53:00 PM,066XX S GREENWOOD AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,20,42.0,14,1184362.0,1861188.0,2024,05/17/2025 03:40:52 PM,41.774269,-87.599710,POINT (-87.599709962 41.774269351)
4,13707823,JJ100020,12/31/2024 11:50:00 PM,012XX N MENARD AVE,0460,BATTERY,SIMPLE,SIDEWALK,False,False,...,29,25.0,08B,1137458.0,1907694.0,2024,05/17/2025 03:40:52 PM,41.902858,-87.770537,POINT (-87.770536741 41.902858242)


In [11]:
df.shape

(258461, 22)

In [34]:
# How to load csv file with spark

#df1 = spark.read.csv('US_crimes_2024.csv', header=True)
#df1.show(5)


**Creating Schemas** Schemas are Structure Types (StructType) ie column name, datatypes and saying weather the column can contain missing value (True) in Spark

In [38]:
# checking the datatype

df.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                  int64
Ward                      int64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [ ]:
# StructType([
#     StructField('ID', StringType, True),
#     StructField('Case Number', StringType, True),
#     StructField('Date', TimestampType, True),
#     StructField('Block', StringType, True),
#     StructField('IUCR', StringType, True),
#     StructField('Primary Type', StringType, True),
#     StructField('Description', StringType, True),
#     StructField('Location Description', StringType, True),
#     StructField('Arrest', StringType, True),
#     StructField('Domestic', BooleanType, True),
#     StructField('District', StringType, True),
#     StructField('Ward', StringType, True),
#     StructField('Community Area', StringType, True),
#     StructField('FBI Code', StringType, True),
#     StructField('X Coordinate', StringType, True),
#     StructField('Y Coordinate', StringType, True),
#     StructField('Year', IntegerType, True),
#     StructField('Updated On', StringType, True),
#     StructField('Latitude', DoubleType, True),
#     StructField('Longitude', DoubleType, True),
#     StructField('Location', StringType, True)
# ])

In [44]:
# Another way to achieve it is like this

labels = [
    ('ID', StringType()),
    ('Case Number', StringType()),
    ('Date', TimestampType()),
    ('Block', StringType()),
    ('IUCR', StringType()),
    ('Primary Type', StringType()),
    ('Description', StringType()),
    ('Location Description', StringType()),
    ('Arrest', StringType()),
    ('Domestic', BooleanType()),
    ('District', StringType()),
    ('Ward', StringType()),
    ('Community Area', StringType()),
    ('FBI Code', StringType()),
    ('X Coordinate', StringType()),
    ('Y Coordinate', StringType()),
    ('Year', IntegerType()),
    ('Updated On', StringType()),
    ('Latitude', DoubleType()),
    ('Longitude', DoubleType()),
    ('Location', StringType())
]

In [52]:
schema = StructType([StructField (x[0],x[1], True) for x in labels])
schema

StructType([StructField('ID', StringType(), True), StructField('Case Number', StringType(), True), StructField('Date', TimestampType(), True), StructField('Block', StringType(), True), StructField('IUCR', StringType(), True), StructField('Primary Type', StringType(), True), StructField('Description', StringType(), True), StructField('Location Description', StringType(), True), StructField('Arrest', StringType(), True), StructField('Domestic', BooleanType(), True), StructField('District', StringType(), True), StructField('Ward', StringType(), True), StructField('Community Area', StringType(), True), StructField('FBI Code', StringType(), True), StructField('X Coordinate', StringType(), True), StructField('Y Coordinate', StringType(), True), StructField('Year', IntegerType(), True), StructField('Updated On', StringType(), True), StructField('Latitude', DoubleType(), True), StructField('Longitude', DoubleType(), True), StructField('Location', StringType(), True)])

In [58]:
df = spark.read.csv('US_crimes_2024.csv', schema=schema)
df.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'boolean'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'int'),
 ('Updated On', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('Location', 'string')]

We have successfully given our dataset the desired schema we wanted

In [72]:
df.limit(5)

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: boolean, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: int, Updated On: string, Latitude: double, Longitude: double, Location: string]

In [70]:
df.show(truncate=False)

+--------+-----------+----+-----------------------------------+----+-------------------+----------------------------------------------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+
|ID      |Case Number|Date|Block                              |IUCR|Primary Type       |Description                                               |Location Description|Arrest|Domestic|District|Ward    |Community Area|FBI Code      |X Coordinate|Y Coordinate|Year   |Updated On|Latitude|Longitude   |Location     |
+--------+-----------+----+-----------------------------------+----+-------------------+----------------------------------------------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+
|ID      |Case Number|NULL|Block                          

**Working with columns**

In [79]:
# Selecting a specific column and number of rows in spark

df.select('Date').show(5)

+----+
|Date|
+----+
|NULL|
|NULL|
|NULL|
|NULL|
|NULL|
+----+
only showing top 5 rows


In [81]:
# Displaying specific columns with number of rows in spark
df.select('ID', 'Case Number', 'Date', 'Arrest').show(4)

+--------+-----------+----+------+
|      ID|Case Number|Date|Arrest|
+--------+-----------+----+------+
|      ID|Case Number|NULL|Arrest|
|13709672|   JJ101940|NULL| false|
|13707925|   JJ100089|NULL|  true|
|13708038|   JJ100035|NULL| false|
+--------+-----------+----+------+
only showing top 4 rows


In [85]:
# Adding a new column to our dataframe called Number with all entries in it as 1

df.withColumn('Number', lit(1)).show(5)

+--------+-----------+----+--------------------+----+-----------------+--------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+------+
|      ID|Case Number|Date|               Block|IUCR|     Primary Type|         Description|Location Description|Arrest|Domestic|District|    Ward|Community Area|      FBI Code|X Coordinate|Y Coordinate|   Year|Updated On|Latitude|   Longitude|     Location|Number|
+--------+-----------+----+--------------------+----+-----------------+--------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+------+
|      ID|Case Number|NULL|               Block|IUCR|     Primary Type|         Description|Location Description|Arrest|    NULL|    Beat|District|          Ward|Community Area|    FBI Code|X Coordinate

In [89]:
# Dropping a column
df = df.drop('Number')
df.show(5)

+--------+-----------+----+--------------------+----+-----------------+--------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+
|      ID|Case Number|Date|               Block|IUCR|     Primary Type|         Description|Location Description|Arrest|Domestic|District|    Ward|Community Area|      FBI Code|X Coordinate|Y Coordinate|   Year|Updated On|Latitude|   Longitude|     Location|
+--------+-----------+----+--------------------+----+-----------------+--------------------+--------------------+------+--------+--------+--------+--------------+--------------+------------+------------+-------+----------+--------+------------+-------------+
|      ID|Case Number|NULL|               Block|IUCR|     Primary Type|         Description|Location Description|Arrest|    NULL|    Beat|District|          Ward|Community Area|    FBI Code|X Coordinate|   NULL|      Year| 